<a href="https://colab.research.google.com/github/filippozuddas/ML-SRT-SETI/blob/main/notebooks/train_CVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install setigen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.2.3
    Uninstalling pyparsing-3.2.3:
      Successfully uninstalled pyparsing-3.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydot 3.0.4 requires pyparsing>=3.0.9, but you have pyparsing 2.4.7 which is incompatible.


In [9]:
import sys
import os

# --- CONFIGURAZIONE PERCORSI ---
PROJECT_ROOT = "/content/filippo/ML-SRT-SETI"

if not os.path.exists(PROJECT_ROOT):
    raise FileNotFoundError(f"{PROJECT_ROOT} folder not found")

os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"✅ Setup completed!")
print(f"   Working directory: {os.getcwd()}")

✅ Setup completed!
   Working directory: /content/filippo/ML-SRT-SETI


In [10]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from src import config, models

# Verifica GPU
print(f"TensorFlow Version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.18.0
Num GPUs Available:  2


In [11]:
# Percorso del file generato nello step precedente
DATA_PATH = "data/processed/train_class0_5k.npy"

# 1. Carica i dati
print(f"Caricamento dati da {DATA_PATH}...")
dataset_cadences = np.load(DATA_PATH)
print(f"Shape originale (Cadenze): {dataset_cadences.shape}")

# 2. Reshape: Da (N_samples, 6, 16, 512, 1) a (N_samples * 6, 16, 512, 1)
# Fondiamo la dimensione delle cadenze con quella dei campioni
# -1 dice a numpy di calcolare automaticamente la dimensione
X_all = dataset_cadences.reshape(-1, config.TCHANS, config.FCHANS_FINAL, 1)

print(f"Shape per il training (Frames singoli): {X_all.shape}")

# 3. Train/Test Split
# Usiamo il random_state per riproducibilità
X_train, X_val = train_test_split(X_all, test_size=0.2, random_state=42)

print(f"Training frames: {X_train.shape}")
print(f"Validation frames: {X_val.shape}")

Caricamento dati da data/processed/train_class0_5k.npy...
Shape originale (Cadenze): (5000, 6, 16, 512, 1)
Shape per il training (Frames singoli): (30000, 16, 512, 1)
Training frames: (24000, 16, 512, 1)
Validation frames: (6000, 16, 512, 1)


In [14]:
print("Verifica integrità dati...")
print(f"Min value: {np.min(X_train)}")
print(f"Max value: {np.max(X_train)}")
print(f"Contains NaNs: {np.isnan(X_train).any()}")
print(f"Contains Infs: {np.isinf(X_train).any()}")

if np.isnan(X_train).any() or np.isinf(X_train).any():
    print("⚠️ ATTENZIONE: Il dataset contiene valori non validi! Il problema è nel preprocessing.")
else:
    print("✅ Dati puliti. Il problema è nell'architettura o nei parametri.")

Verifica integrità dati...
Min value: -5.95879323036152
Max value: 20.552189848819005
Contains NaNs: False
Contains Infs: False
✅ Dati puliti. Il problema è nell'architettura o nei parametri.


In [18]:
# MirroredStrategy copia il modello su tutte le GPU disponibile e divide il batch
strategy = tf.distribute.MirroredStrategy()

# Parametri
input_shape = (config.TCHANS, config.FCHANS_FINAL, 1)
latent_dim = config.LATENT_DIM # 8


with strategy.scope():
  # Costruzione Encoder e Decoder
  encoder = models.build_encoder(input_shape, latent_dim)
  decoder = models.build_decoder(latent_dim, target_shape=input_shape)

  # 2. CVAE Completo
  # beta=1.0 è un VAE standard.
  # beta > 1 forza una rappresentazione latente più "districata" (disentangled)
  cvae = models.CVAE(encoder, decoder, beta=1)

  # Compile
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  cvae.compile(optimizer=optimizer)

# (Opzionale) Visualizza sommario encoder
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 16, 512,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 8, 256,    │        320 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 256,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 4, 128,    │     18,496 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 128,    │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 2, 64,     │     73,856 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 64,     │        512 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 16384)     │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │  2,097,280 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 8)         │      1,032 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 8)         │      1,032 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling_3          │ (None, 8)         │          0 │ z_mean[0][0],     │
│ (Sampling)          │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,192,912 (8.37 MB)

 Trainable params: 2,192,464 (8.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [19]:
# Configurazione Training
EPOCHS = 20
BATCH_SIZE = config.BATCH_SIZE # 32

print("Avvio training...")

history = cvae.fit(
    X_train,
    validation_data=(X_val, X_val), # Nel VAE input e target sono uguali (ricostruzione)
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True
)

print("Training completato.")

Avvio training...
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - kl_loss: nan - loss: nan - reconstruction_loss: nan

ValueError: No loss to compute. Provide a `loss` argument in `compile()`.